# Profiling and Timing Code   <a href="https://colab.research.google.com/github/Ahmad-Zaki/Python-Notes/blob/main/Timing%20and%20Profiling%20Python%20code/timing-and-profiling.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

## Introduction <a class="anchor" id="introduction"></a>

When you Create a function to do some task, there are often different implementations you can use to get the task done. Therefore, it can be useful sometimes to dig into the performance of each implementation to see how it performs or if it can be optimized in any way. Sometimes it's useful to check the execution time of a given command or set of commands; other times it's useful to measure a multiline process and determine where the bottleneck lies in a series of operations. 

IPython provides different commands for timing and profiling your code. We'll go over the following IPython magic commands:

- ``%time``: Time the execution of a single statement
- ``%timeit``: Time repeated execution of a single statement for more accuracy
- ``%prun``: Run code with the profiler
- ``%lprun``: Run code with the line-by-line profiler
- ``%memit``: Measure the memory use of a single statement
- ``%mprun``: Run code with the line-by-line memory profiler

The last three commands are not bundled with IPython–you'll need to get the ``line_profiler`` and ``memory_profiler`` extensions, which we will discuss in the following sections.

## Timing Your Code

We can use ``%timeit`` line-magic and ``%%timeit`` cell-magic  to measure the average time it takes to execute a snippet of code:

In [1]:
%timeit sum(range(100))

1.32 µs ± 8.31 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


The output tells us that the execution of this line takes in average $1.31 \mu s$ with a standard deviation of $11.2 ns$. These statistics are calculated by measuring the time it takes to run this line of code 1 million times, repeated 7 times.

Note that because this line takes very little time to execute, ``%timeit`` automatically does a large number of repetitions.
For slower commands, ``%timeit`` will automatically adjust and perform fewer repetitions:

In [2]:
%%timeit
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

385 ms ± 12.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now that the code block takes relatively longer time to execute, ``%timeit`` had adjusted to measure the time of only a single run, repeated 7 times.

If we want to fix the number of runs or loops, we can set the number of loops (``-n``) and/or runs (``-r``) as flags for ``%timeit``:

In [3]:
# Set the number of runs to 2 (-r2).
# Set the number of loops to 10 (-n10).

%timeit -r2 -n10 [i for i in range(1000000)]

74.4 ms ± 1.31 ms per loop (mean ± std. dev. of 2 runs, 10 loops each)


Also, we can save the result of each run in a variable using the flag (``-o``):

In [4]:
times = %timeit -o a = sum(range(10000))
print(times.timings)
print(f"Best time: {times.best}")
print(f"Worst time: {times.worst}")

189 µs ± 4.36 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
[0.00018676577999999892, 0.0001876240100000018, 0.00018658344000000113, 0.00019165486999999927, 0.00019916715000000237, 0.00018653177999999854, 0.0001868585299999978]
Best time: 0.00018653177999999854
Worst time: 0.00019916715000000237


If you want to know the other available options for ``%timeit``, you can IPython help functionality (i.e., type ``%timeit?`` at the IPython prompt).

Sometimes repeating an operation is not the best option.
For example, if we have a list that we'd like to sort, we might be misled by a repeated operation.
Sorting a pre-sorted list is much faster than sorting an unsorted list, so the repetition will skew the result:

In [5]:
import random
L = [random.random() for i in range(100000)]
%timeit L.sort()

1.22 ms ± 291 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


For this, the ``%time`` magic function may be a better choice. It also is a good choice for longer-running commands, when short, system-related delays are unlikely to affect the result.
Let's time the sorting of an unsorted and a presorted list:

In [6]:
import random
L = [random.random() for i in range(100000)]
print("sorting an unsorted list:")
%time L.sort()
print("="*20)
print("sorting an already sorted list:")
%time L.sort()

sorting an unsorted list:
Wall time: 22 ms
sorting an already sorted list:
Wall time: 2 ms


We can notice that ``%time`` takes much longer time versus ``%timeit``, even for the presorted list. This is because ``%timeit`` will use the best available timing function on your system. Also, it prevents python from calling the garbage collector so that it doesn't interfere with the timing. another major difference is that ``%timeit`` makes it trivial to time the execution of the function for thousands of iterations (which is the only time where a timing result is meaningful). This decreases the importance of a single run taking longer than the others (e.g. due to your system resources being hogged by some other program).

Just like ``%%timeit``, we can use ``%%time`` to time multiline scripts: 

In [7]:
%%time
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

Wall time: 560 ms


If you want to know more information about ``%time``, you can IPython help functionality (i.e., type ``%time?`` at the IPython prompt).

## Profiling Code Blocks

Sometimes it is more helpful to time commands in context instead of timing them on their own, Python contains a built-in code profiler (which you can read about in the Python documentation), but IPython offers a much more convenient way to use this profiler, in the form of the magic function ``%prun``.

Lets define a function that does some calculations:

In [8]:
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
    return total

Now we can call ``%prun`` with a function call to see the profiled results:

In [9]:
%prun sum_of_lists(1000000)

         14 function calls in 1.084 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.905    0.181    0.905    0.181 3519952779.py:4(<listcomp>)
        5    0.128    0.026    0.128    0.026 {built-in method builtins.sum}
        1    0.040    0.040    1.073    1.073 3519952779.py:1(sum_of_lists)
        1    0.011    0.011    1.084    1.084 <string>:1(<module>)
        1    0.000    0.000    1.084    1.084 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

The result is a table that indicates, in order of total time on each function call, where the execution is spending the most time. In this case, the bulk of execution time is in the list comprehension inside ``sum_of_lists``.
From here, we could start thinking about what changes we might make to improve the performance in the algorithm.

## Line by Line Profiling

The profiling of ``%prun`` is very handy, but it would be more convenient if we can get a line-by-line profiling report. Unfortunately, this isn't built into Python or IPython. However, we can still get it by installing the ``line_profiler`` package using the following command:

In [10]:
!pip install line_profiler

Now we can load the ``line_profiler`` IPython extention provided in this package:

In [11]:
%load_ext line_profiler

Now the ``%lprun`` command will do a line-by-line profiling of any function–in this case, we need to tell it explicitly which functions we're interested in profiling:

In [12]:
%lprun -f sum_of_lists sum_of_lists(1000000)

Timer unit: 1e-07 s

Total time: 1.96307 s
File: C:\Users\DesTr\AppData\Local\Temp/ipykernel_18004/3519952779.py
Function: sum_of_lists at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def sum_of_lists(N):
     2         1         23.0     23.0      0.0      total = 0
     3         6        284.0     47.3      0.0      for i in range(5):
     4         5   18464399.0 3692879.8     94.1          L = [j ^ (j >> i) for j in range(N)]
     5         5    1165958.0 233191.6      5.9          total += sum(L)
     6         1          7.0      7.0      0.0      return total

The first line gives us the key to reading the values in the table: the time is reported in $0.1\mu s$. The total execution time is $1.98s$. We can see how many times each line is executed, its total runtime, and its percentage of the total time. Now we can know exactly what line takes the most time to execute, and decide what to change to improve the timing of the code.

Although code profilers like ``%prun`` and ``%lprun`` provide very useful information, they measure the execution time only once. this may deliver inconsistent results depending the condition of your system when you run them. On the contrary, ``%timeit`` performs multible runs to get a more robust result for the time it takes to execute a statement or a script, but it is more time consuming for larger scripts with complex statements.

For more information on ``%prun`` or ``%lprun``, as well as its available options, use the IPython help functionality (i.e., type ``%lprun?`` at the IPython prompt).

# Memory Profiling

Another aspect of measuring our code performance is the amount of memory it takes during runtime. Just like ``line_profiler``, we can install ``memory_profiler`` using pip:

In [13]:
%pip install memory_profiler

Note: you may need to restart the kernel to use updated packages.


Now we can use IPython to load the extension:

In [14]:
%load_ext memory_profiler

The memory profiler extension contains two useful magic functions: the ``%memit`` magic (which offers a memory-measuring equivalent of ``%timeit``) and the ``%mprun`` function (which offers a memory-measuring equivalent of ``%lprun``).

The ``%memit`` function can be used rather simply:

In [15]:
%memit sum_of_lists(1000000)

peak memory: 161.12 MiB, increment: 67.38 MiB


We see that this function uses about 161 MB of memory.

For a line-by-line description of memory use, we can use the ``%mprun`` magic.
Unfortunately, this magic works only for functions defined in separate modules rather than the notebook itself, so we'll start by using the ``%%file`` magic to create a simple module called ``mprun_demo.py``, which contains our ``sum_of_lists`` function, with one addition that will make our memory profiling results more clear:

In [16]:
%%file mprun_demo.py
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
        del L # remove reference to L
    return total

Overwriting mprun_demo.py


We can now import the new version of this function and run the memory line profiler:

In [17]:
from mprun_demo import sum_of_lists
%mprun -f sum_of_lists sum_of_lists(500000)

Filename: d:\Workspace\Timing and Profiling Python code\mprun_demo.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     1     94.1 MiB     94.1 MiB           1   def sum_of_lists(N):
     2     94.1 MiB      0.0 MiB           1       total = 0
     3     95.8 MiB     -1.0 MiB           6       for i in range(5):
     4    112.6 MiB -14877566.9 MiB     2500015           L = [j ^ (j >> i) for j in range(N)]
     5    112.6 MiB     -0.0 MiB           5           total += sum(L)
     6     95.8 MiB    -72.2 MiB           5           del L # remove reference to L
     7     95.5 MiB     -0.3 MiB           1       return total

the Increment column tells us how much each line affects the total memory budget: observe that when we create and delete the list ``L``, we are adding and removing about 25 MB of memory usage.
This is on top of the background memory usage from the Python interpreter itself.

For more information on ``%memit`` and ``%mprun``, as well as their available options, use the IPython help functionality (i.e., type ``%memit?`` at the IPython prompt).